<a href="https://colab.research.google.com/github/chetools/CHE4071_Spring2025/blob/main/ITAE_PID_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 689.8 kB/s eta 0:00:00


In [66]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift, rhs2TF

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import sympy
import scipy as sp
import scipy.signal as sig
from scipy.optimize import minimize

In [52]:
def foptd(t, K, tau, theta):
    return np.where(t>theta,K*(1-np.exp(-(t-theta)/tau)), 0)

In [99]:
Gp = 1.5/(((7*s+1)**3)*(12*s+1)**2)

In [81]:
t, y = sim(Gp, lambda t: 1.2, N=50, dt=2.5)
y_data = np.random.normal(y, 0.05)
y_model = foptd(t, 1.8, 30, 15)

In [70]:
def sse(v):
    KA, tau, theta = v
    return np.sum((y_data - foptd(t, KA, tau, theta))**2)

In [71]:
v0=np.array([1.8, 30, 15])
KA, tau, theta = minimize(sse, v0).x

In [73]:
fig = make_subplots()
fig.add_scatter(x=t, y=y_data, mode='markers')
fig.add_scatter(x=t, y=foptd(t,KA,tau,theta), mode='lines')
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [77]:
PA = 0.586
PB = -0.916
IA = 1.03
IB = -0.165
K = KA/1.2
Y = PA*(theta/tau)**PB
Kc = Y/K
taui = tau/(IA + IB*(theta/tau))

In [97]:
Gc = Kc*(1 + 1/(s*taui))
Gcl = Gc*Gp/(1+Gc*Gp)

In [98]:
t, y_servo = sim(Gcl, lambda t: 1., N=1000, dt=0.5)
fig2= make_subplots()
fig2.add_scatter(x=t, y=y_servo, mode='lines')
fig2.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)